In [1]:
import json
import os
from copy import deepcopy
from typing import List, Tuple

In [2]:
hypothesis = 'patch_attention'

In [3]:
base_configuration = {
    'hypothesis': hypothesis,
    'experiment': None,  # will change below
    'data': {
        'path': '/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/data_for_training/03_04_with_pesto_and_coord/folds_training_dicts.pkl',
        'use_pesto': True,
        'ablation_string': '111111111',
        'max_folds': None,
        'use_coordinates': True
    },
    'model': {
        'name': 'patch_attention',
        'kwargs': {
            'output_mlp_hidden_sizes': None, 
            'output_mlp_dropout_rate': 0.25,
            'attention_mlp_hidden_sizes': None, 
            'attention_mlp_dropout_rate': 0.25,
            'activation': 'relu',
            'input_shape': (10, 9),
            'max_number_of_patches': 10,
            'attention_dimension': None,
            'num_heads': None
        }
    },
    'compile': {
        'optimizer': {
            'name': 'adamw',
            'kwargs': {'learning_rate': 1e-3}
        },
        'loss': {
            'name': 'binary_cross_entropy',
            'kwargs': {}
        }
    },
    'fit': {
        'epochs': 200,
        'batch_size': 256,
        'verbose': 1,
        'n_early_stopping_epochs': 12
    }
}

In [4]:
def mlps_option_to_str(mlps_option: List[Tuple[int, int]]) -> str:
    res = f'{str(mlps_option[0][0])}-{str(mlps_option[0][1])}'
    return f'{len(mlps_option)}_{res}'

In [5]:
def option_to_name(option: dict) -> str:
    name = f'attdim_{option["attention_dimension"]}-heads_{option["num_heads"]}-a_{mlps_option_to_str(option["attention_mlp_hidden_sizes"])}-o_{mlps_option_to_str(option["output_mlp_hidden_sizes"])}'
    return name

In [6]:
options = [
    {
        'output_mlp_hidden_sizes': [(256, 256)], 
        'attention_mlp_hidden_sizes': [(256, 256)], 
        'attention_dimension': 256,
        'num_heads': 8
    },
    {
        'output_mlp_hidden_sizes': [(1024, 256)], 
        'attention_mlp_hidden_sizes': [(1024, 256)], 
        'attention_dimension': 256,
        'num_heads': 8
    },
    {
        'output_mlp_hidden_sizes': [(2048, 512)], 
        'attention_mlp_hidden_sizes': [(2048, 512)], 
        'attention_dimension': 512,
        'num_heads': 8
    },
    {
        'output_mlp_hidden_sizes': [(1024, 256), (1024, 256)], 
        'attention_mlp_hidden_sizes': [(1024, 256), (1024, 256)], 
        'attention_dimension': 256,
        'num_heads': 8
    },
    {
        'output_mlp_hidden_sizes': [(1024, 256)], 
        'attention_mlp_hidden_sizes': [(1024, 256), (1024, 256)], 
        'attention_dimension': 256,
        'num_heads': 8
    },
    {
        'output_mlp_hidden_sizes': [(1024, 256), (1024, 256)], 
        'attention_mlp_hidden_sizes': [(1024, 256)], 
        'attention_dimension': 256,
        'num_heads': 8
    },
    {
        'output_mlp_hidden_sizes': [(1024, 256)], 
        'attention_mlp_hidden_sizes': [(1024, 256)], 
        'attention_dimension': 256,
        'num_heads': 1
    },
    {
        'output_mlp_hidden_sizes': [(1024, 256)], 
        'attention_mlp_hidden_sizes': [(1024, 256)], 
        'attention_dimension': 256,
        'num_heads': 4
    },
    ]

In [7]:
dir_path = f'../data/{hypothesis}'
os.makedirs(dir_path, exist_ok=True)

for option in options:
    configuration = deepcopy(base_configuration)
    configuration['model']['kwargs'].update(option)
    
    output_mlp_str = mlps_option_to_str(option['output_mlp_hidden_sizes'])
    attention_mlp_str = mlps_option_to_str(option['attention_mlp_hidden_sizes'])
    
    name = option_to_name(option)
    configuration['experiment'] = name
    
    with open(f'{dir_path}/{name}.json', 'w') as f:
        json.dump(configuration, f, indent=4)
    